In [1]:
# タイタニック生存予測

In [2]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/kaggle/titanic

/content/drive/MyDrive/kaggle/titanic


In [4]:
import pandas as pd
import numpy as np

In [5]:
# データの読み込み
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [6]:
# データの先頭の５行を表示
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# データの型を調べる
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [8]:
# nullかどうかを調べてから、その数を調べる
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
test.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [10]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
x = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = train[['Survived']]

In [12]:
# one-hotエンコーディングをする
# 展開してそれぞれ0か１を入れる
x = pd.get_dummies(x, columns=['Pclass', 'Sex','Embarked'])
x

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,35.0,0,0,8.0500,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,0,0,13.0000,0,1,0,0,1,0,0,1
887,19.0,0,0,30.0000,1,0,0,1,0,0,0,1
888,NaN,1,2,23.4500,0,0,1,1,0,0,0,1
889,26.0,0,0,30.0000,1,0,0,0,1,1,0,0


In [13]:
# nullの数を調べる
x['Age'].isnull().sum()

177

In [14]:
# nullを埋める
x['Age'] = x['Age'].fillna(x['Age'].mean())
# 埋まったか確認
x['Age'].isnull().sum()

0

In [15]:
# 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [16]:
# knnのインポート
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [17]:
knn.fit(x_train, y_train)
knn.score(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


0.8158682634730539

In [18]:
knn.score(x_test, y_test)

0.7354260089686099

In [19]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [20]:
knn.fit(x_train_scaled, y_train)
knn.score(x_train_scaled, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


0.8667664670658682

In [21]:
knn.fit(x_test_scaled, y_test)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [22]:
# submit用に作る
x_for_submit = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
submit = test[['PassengerId']]
x_for_submit['Age']  = x_for_submit['Age'].fillna(x_for_submit['Age'].mean())
x_for_submit['Fare'] = x_for_submit['Fare'].fillna(x_for_submit['Fare'].mean())
x_for_submit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.50000,0,0,7.8292,Q
1,3,female,47.00000,1,0,7.0000,S
2,2,male,62.00000,0,0,9.6875,Q
3,3,male,27.00000,0,0,8.6625,S
4,3,female,22.00000,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,30.27259,0,0,8.0500,S
414,1,female,39.00000,0,0,108.9000,C
415,3,male,38.50000,0,0,7.2500,S
416,3,male,30.27259,0,0,8.0500,S


In [23]:
x_for_submit = pd.get_dummies(x_for_submit, columns=['Pclass', 'Sex','Embarked'])
x_for_submit

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,34.50000,0,0,7.8292,0,0,1,0,1,0,1,0
1,47.00000,1,0,7.0000,0,0,1,1,0,0,0,1
2,62.00000,0,0,9.6875,0,1,0,0,1,0,1,0
3,27.00000,0,0,8.6625,0,0,1,0,1,0,0,1
4,22.00000,1,1,12.2875,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1
414,39.00000,0,0,108.9000,1,0,0,1,0,1,0,0
415,38.50000,0,0,7.2500,0,0,1,0,1,0,0,1
416,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1


In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_for_submit)
x_for_submit = scaler.transform(x_for_submit)
x_for_submit

array([[ 0.3349926 , -0.49947002, -0.4002477 , ..., -0.56814154,
         2.84375747, -1.35067551],
       [ 1.32553003,  0.61699237, -0.4002477 , ..., -0.56814154,
        -0.35164743,  0.74037028],
       [ 2.51417495, -0.49947002, -0.4002477 , ..., -0.56814154,
         2.84375747, -1.35067551],
       ...,
       [ 0.65196458, -0.49947002, -0.4002477 , ..., -0.56814154,
        -0.35164743,  0.74037028],
       [ 0.        , -0.49947002, -0.4002477 , ..., -0.56814154,
        -0.35164743,  0.74037028],
       [ 0.        ,  0.61699237,  0.61989583, ...,  1.76012477,
        -0.35164743, -1.35067551]])

In [25]:
submit['Survived'] = knn.predict(x_for_submit)
submit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [26]:
# csvファイルを作成
submit.to_csv('submission/submit03.csv', index=False)

In [27]:
# ニューラルネットを試す
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [28]:
model = keras.Sequential()
model.add(Dense(20, input_dim=12))
model.add(Activation('relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [29]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train_scaled, y_train, epochs=1000, verbose=2)

Epoch 1/1000
21/21 - 1s - loss: 0.7217 - accuracy: 0.5329
Epoch 2/1000
21/21 - 0s - loss: 0.6485 - accuracy: 0.6602
Epoch 3/1000
21/21 - 0s - loss: 0.5936 - accuracy: 0.7290
Epoch 4/1000
21/21 - 0s - loss: 0.5540 - accuracy: 0.7605
Epoch 5/1000
21/21 - 0s - loss: 0.5220 - accuracy: 0.7695
Epoch 6/1000
21/21 - 0s - loss: 0.4978 - accuracy: 0.7740
Epoch 7/1000
21/21 - 0s - loss: 0.4790 - accuracy: 0.7769
Epoch 8/1000
21/21 - 0s - loss: 0.4650 - accuracy: 0.7844
Epoch 9/1000
21/21 - 0s - loss: 0.4548 - accuracy: 0.7979
Epoch 10/1000
21/21 - 0s - loss: 0.4464 - accuracy: 0.8069
Epoch 11/1000
21/21 - 0s - loss: 0.4393 - accuracy: 0.8114
Epoch 12/1000
21/21 - 0s - loss: 0.4334 - accuracy: 0.8174
Epoch 13/1000
21/21 - 0s - loss: 0.4291 - accuracy: 0.8204
Epoch 14/1000
21/21 - 0s - loss: 0.4246 - accuracy: 0.8263
Epoch 15/1000
21/21 - 0s - loss: 0.4224 - accuracy: 0.8293
Epoch 16/1000
21/21 - 0s - loss: 0.4190 - accuracy: 0.8323
Epoch 17/1000
21/21 - 0s - loss: 0.4155 - accuracy: 0.8368
Epoch 

In [ ]:
# テストデータで評価
model.fit(x_test_scaled, y_test, epochs=1000, verbose=2)

Epoch 1/1000
7/7 - 0s - loss: 0.7308 - accuracy: 0.7803
Epoch 2/1000
7/7 - 0s - loss: 0.6667 - accuracy: 0.7937
Epoch 3/1000
7/7 - 0s - loss: 0.6038 - accuracy: 0.8117
Epoch 4/1000
7/7 - 0s - loss: 0.5810 - accuracy: 0.8206
Epoch 5/1000
7/7 - 0s - loss: 0.5542 - accuracy: 0.8027
Epoch 6/1000
7/7 - 0s - loss: 0.5231 - accuracy: 0.8206
Epoch 7/1000
7/7 - 0s - loss: 0.5076 - accuracy: 0.8161
Epoch 8/1000
7/7 - 0s - loss: 0.4888 - accuracy: 0.8206
Epoch 9/1000
7/7 - 0s - loss: 0.4731 - accuracy: 0.8251
Epoch 10/1000
7/7 - 0s - loss: 0.4571 - accuracy: 0.8341
Epoch 11/1000
7/7 - 0s - loss: 0.4447 - accuracy: 0.8296
Epoch 12/1000
7/7 - 0s - loss: 0.4315 - accuracy: 0.8341
Epoch 13/1000
7/7 - 0s - loss: 0.4169 - accuracy: 0.8386
Epoch 14/1000
7/7 - 0s - loss: 0.4051 - accuracy: 0.8430
Epoch 15/1000
7/7 - 0s - loss: 0.3959 - accuracy: 0.8386
Epoch 16/1000
7/7 - 0s - loss: 0.3825 - accuracy: 0.8475
Epoch 17/1000
7/7 - 0s - loss: 0.3733 - accuracy: 0.8475
Epoch 18/1000
7/7 - 0s - loss: 0.3628 - 

In [ ]:
# submit
x_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
submit = test[['PassengerId']]

x_test = pd.get_dummies(x_test, columns=['Pclass', 'Sex','Embarked'])

# Fareのnullを埋める
# nullにFareの平均値を埋め込む
x_test['Fare'] = x_test['Fare'].fillna(x_test['Fare'].mean())
# 標準化
scaler = StandardScaler()
scaler.fit(x_test)
x_test_scaled = scaler.transform(x_test)

submit['Survived'] = model.predict(x_test_scaled)
submit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,PassengerId,Survived
0,892,1.489918e-09
1,893,2.235663e-02
2,894,9.999997e-01
3,895,3.525227e-03
4,896,2.894104e-04
...,...,...
413,1305,NaN
414,1306,1.000000e+00
415,1307,7.241160e-05
416,1308,NaN


In [ ]:
# nullを0で埋める
submit['Survived'] = submit['Survived'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 0に置き換えられたか確認
submit['Survived']

0      1.489918e-09
1      2.235663e-02
2      9.999997e-01
3      3.525227e-03
4      2.894104e-04
           ...     
413    0.000000e+00
414    1.000000e+00
415    7.241160e-05
416    0.000000e+00
417    0.000000e+00
Name: Survived, Length: 418, dtype: float32

In [ ]:
# one hot エンコーディングしたい

for s in submit['Survived']:
  if s > 0.5:
    print('1')
    s = 1
  elif s <= 0.5:
    print('0')
    s = 0

submit.head

0
0
1
0
0
1
0
0
1
0
0
0
1
0
1
1
0
0
0
0
1
1
0
1
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
1
0
1
0
0
1
1
0
0
0
0
0
0
0
1
0
0
1
0
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
1
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
1
0
1
0
1
0
0
0
0
0
0
1
1
0
1
0
0
0
0
1
1
1
0
0
0
0
1
0
0
0
0
1
0
1
0
1
1
0
1
0
1
0
1
0
0
0
0
0
0
1
0
0
1
0
1
0
1
1
1
1
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
1
0
0
0
1
0
1
0
0
0
0
1
1
0
1
0
0
1
1
0
0
1
0
0
1
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
1
1
1
0
0
1
0
1
0
0
1
0
1
1
1
0
0
1
0
1
0
1
0
0
1
0
0
0


<bound method NDFrame.head of      PassengerId      Survived
0            892  1.489918e-09
1            893  2.235663e-02
2            894  9.999997e-01
3            895  3.525227e-03
4            896  2.894104e-04
..           ...           ...
413         1305  0.000000e+00
414         1306  1.000000e+00
415         1307  7.241160e-05
416         1308  0.000000e+00
417         1309  0.000000e+00

[418 rows x 2 columns]>

In [ ]:
# one hot エンコーディングしたい

for i in range(len(submit['Survived'])):
  if submit['Survived'][i] > 0.5:
    submit['Survived'][i] = 1
  elif submit['Survived'][i] <= 0.5:
    submit['Survived'][i] = 0

submit['Survived']
submit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,1.0
3,895,0.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [ ]:
# csvファイルを作成
submit.to_csv('submission/submit06.csv', index=False)